# Analysis
This notebook is used to analyze sound files and extract spectra, harmonics, formant, etc.

In [54]:
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import sounddevice as sd
import scipy
import pickle
%matplotlib qt5

In [ ]:
# helpers to play/write/read audio
fs=44100

def play_audio(y):
    sd.play(y,fs)

def write_audio(y, filename):
    sf.write(filename+'.wav',y,fs)

def read_audio(filename):
    y, fs = sf.read(filename +'.wav')
    return y

def graph_signal(y, start=None, end=None):    
    plt.figure()
    x = np.arange(len(y))/fs
    plt.plot(x, y)
    plt.xlim(start, end)

def graph_spectrum(y, distance, title=""):
    Y = scipy.fft.rfft(y)
    plt.figure()
    Y = abs(Y)
    peaks, _ = scipy.signal.find_peaks(Y, distance=distance)
    plt.title(title)
    plt.plot(Y)
    plt.plot(peaks, Y[peaks], "x")

    # also graph multiples of the fundamental frequency
    space = peaks[1]
    a = (np.arange(20) + 1) * space
    plt.plot(a, np.zeros_like(a), '2', color='red')

# convert to mono by taking left channel (arbitrary)
def to_mono(y):
    return y[:,0]


## Visualize Vowel Formants

In [52]:
distance = 300
for v in ['a', 'e', 'i', 'o', 'u']:
    y = read_audio(f"./samples/vowel/{v}")
    y = to_mono(y)
    graph_spectrum(y, distance, title=v)

From the graphs visualized above, we can see each vowel has different levels in harmonics. The harmonics die off after the 15th harmonic, so we will just use the first 17 harmonics. A few of the vowels have a noticable second formant around the 14/15th harmonic.

Note that the peaks returned by findpeaks include the 0th harmonic, which we ignore.

In [ ]:
harmonic_levels = {}
num_harmonics = 17

for v in ['a', 'e', 'i', 'o', 'u']:
    y = read_audio(f"./samples/vowel/{v}")
    y = to_mono(y)
    Y = scipy.fft.rfft(y)
    Y = abs(Y)
    peaks, _ = scipy.signal.find_peaks(Y, distance=distance)
    peaks = peaks[1:(num_harmonics + 1)]

    levels = Y[peaks]
    harmonic_levels[v] = levels/levels[0]

with open('vowel_harmonics', 'wb') as f:
    pickle.dump(harmonic_levels, f)
print("Wrote vowel harmonic levels to file 'harmonic_levels'.")

print(harmonic_levels)

{'a': array([1.        , 0.44224491, 0.2709633 , 0.14451922, 0.15657546,
       0.05711973, 0.17993182, 0.13086639, 0.02072282, 0.03268279,
       0.01449394, 0.00599529, 0.00925251, 0.04949498, 0.00910186,
       0.00193493, 0.00224668]), 'e': array([1.00000000e+00, 7.09075375e-01, 2.91944480e-01, 8.93113038e-02,
       8.42916607e-02, 2.53495025e-02, 1.59266422e-02, 5.83055033e-02,
       9.50779930e-02, 1.90206251e-02, 1.37401617e-02, 1.11471728e-02,
       2.45177226e-03, 1.62763021e-02, 7.14625822e-03, 9.97373863e-04,
       7.38126027e-04]), 'i': array([1.00000000e+00, 3.58734949e-01, 5.67327682e-02, 2.00292439e-02,
       7.20194663e-03, 1.00627698e-02, 4.47768903e-03, 1.16749850e-03,
       1.64585275e-03, 1.38811689e-03, 8.99514854e-04, 1.20915894e-02,
       2.96753568e-02, 1.99762786e-02, 5.35037203e-03, 3.38242981e-03,
       9.54795803e-04]), 'o': array([1.00000000e+00, 3.75353181e-01, 2.09521668e-01, 8.07667935e-02,
       5.81531304e-02, 9.19821977e-02, 9.53650642e-03, 9